# Homework

In [1]:
import requests
import dlt
import json

from pathlib import Path
from dlt.destinations import qdrant

## Question 1. dlt Version

In [3]:
!pip show dlt | grep Version

Version: 1.12.3


## dlt Resourse

In [7]:
@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

## Question 2. dlt pipeline

In [9]:
qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [10]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

/Users/markogolovko/Projects/llm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:02<00:00,  2.03it/s]


Run started at 2025-07-07 09:35:20.459979+00:00 and COMPLETED in 8.43 seconds with 4 steps.
Step extract COMPLETED in 1.02 seconds.

Load package 1751880925.084119 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.04 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1751880925.084119 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 2.74 seconds.
Pipeline zoomcamp_pipeline load step completed in 2.73 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /Users/markogolovko/Projects/llm/dlt/db.qdrant location to store data
Load package 1751880925.084119 is LOADED and contains no failed jobs

Step run COMPLETED in 8.42 seconds.
Pipeline zoomcamp_pipeline load step completed in 2.73 seconds
1 load package(s) were loaded to destinati

## Question 3. Embeddings

In [9]:
meta_path = Path("db.qdrant/meta.json")
with open(meta_path, "r", encoding="utf-8") as f:
    meta = json.load(f)
for collection_name, collection_info in meta["collections"].items():
    vectors = collection_info.get("vectors", {})
    for vector_name in vectors:
        print(f"  → Embedding model (vector name): {vector_name}")

    break

  → Embedding model (vector name): fast-bge-small-en
